In [25]:
path='C:/임시/RData/example/'

In [26]:
import keras
import numpy as np
import cv2
 
np.random.seed(15)

In [27]:
generator =ImageDataGenerator(rotation_range = 20,
                              width_shift_range = 0.2,
                              height_shift_range = 0.2,
                              rescale = 1. / 255)

In [28]:
batch_size = 4 # 한번에 4장씩 이미지 읽어옴
iterations = 5 # 5번 증식을 진행
images = []

In [29]:
obj = generator.flow_from_directory(
    path,
    target_size = (150, 150),
    batch_size = batch_size,
    class_mode = 'binary')

Found 4 images belonging to 2 classes.


In [30]:
for i in enumerate(range(iterations)):
    img, label = obj .next()
    n_img = len(label)
    base=cv2.cvtColor(img[0],cv2.COLOR_RGB2BGR)
    for idx in range(n_img - 1):
        img2 = cv2.cvtColor(img[idx + 1], cv2.COLOR_RGB2BGR)
        base = np.hstack((base, img2))
    images.append(base)

img = images[0]
for idx in range(len(images) - 1):
    img = np.vstack((img, images[idx + 1]))


In [31]:
cv2.imshow('result', img)
cv2.waitKey()
cv2.destroyAllWindows()

In [32]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils

In [33]:
model = Sequential()

model.add(Conv2D(2, (3, 3), padding='same', activation='relu', input_shape=(8, 8, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(3, (2, 2), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='softmax'))


In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 8, 8, 2)           20        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 4, 2)           0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 4, 3)           27        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 2, 3)           0         
_________________________________________________________________
flatten (Flatten)            (None, 12)                0         
_________________________________________________________________
dense (Dense)                (None, 8)                 104       
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 2

In [44]:
train_datagen = ImageDataGenerator(rescale=1./255)

In [45]:
# C:\임시\RData\hand\handwriting_shape
train_generator = train_datagen.flow_from_directory(
    'C:/임시/RData/hand/handwriting_shape/train',
    target_size=(24,24),
    batch_size=3, # 한 폴더에서 1장씩 총 3장 이미지를 가져와서 증식
    class_mode='categorical'
)

Found 45 images belonging to 3 classes.


In [46]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [47]:
test_generator = test_datagen.flow_from_directory(
    'C:/임시/RData/hand/handwriting_shape/test',
    target_size=(24,24),
    batch_size=3,
    class_mode='categorical'
)

Found 15 images belonging to 3 classes.


In [48]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(24,24,3)))

model.add(Conv2D(64, (3, 3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dense(3, activation='softmax'))

In [40]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 22, 22, 32)        896       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               819328    
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 839,107
Trainable params: 839,107
Non-trainable params: 0
________________________________________________

In [49]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [50]:
model.fit_generator(train_generator,
                   steps_per_epoch=15,
                   epochs=50,
                    validation_data=test_generator,validation_steps=5)

Epoch 1/50
15/15 [==============================] - 0s 21ms/step - loss: 1.0123 - accuracy: 0.6444 - val_loss: 0.6704 - val_accuracy: 1.0000
Epoch 2/50
15/15 [==============================] - 0s 13ms/step - loss: 0.3278 - accuracy: 0.9333 - val_loss: 0.1778 - val_accuracy: 0.9333
Epoch 3/50
15/15 [==============================] - 0s 12ms/step - loss: 0.0907 - accuracy: 0.9778 - val_loss: 0.0380 - val_accuracy: 1.0000
Epoch 4/50
15/15 [==============================] - 0s 12ms/step - loss: 0.0153 - accuracy: 1.0000 - val_loss: 0.0307 - val_accuracy: 1.0000
Epoch 5/50
15/15 [==============================] - 0s 12ms/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.0084 - val_accuracy: 1.0000
Epoch 6/50
15/15 [==============================] - 0s 12ms/step - loss: 9.2892e-04 - accuracy: 1.0000 - val_loss: 0.0077 - val_accuracy: 1.0000
Epoch 7/50
15/15 [==============================] - 0s 12ms/step - loss: 5.3778e-04 - accuracy: 1.0000 - val_loss: 0.0086 - val_accuracy: 1.0000
Epoch

In [51]:
model.evaluate_generator(test_generator,steps=5)

Instructions for updating:
Please use Model.evaluate, which supports generators.


[7.249514601426199e-05, 1.0]

In [52]:
output = model.predict_generator(test_generator,steps=5)

Instructions for updating:
Please use Model.predict, which supports generators.


In [53]:
output

array([[1.0000000e+00, 1.2345122e-08, 1.2328461e-12],
       [3.9962857e-04, 9.9957138e-01, 2.9009727e-05],
       [1.0491691e-11, 9.8199440e-09, 1.0000000e+00],
       [2.3964869e-08, 1.1461838e-04, 9.9988532e-01],
       [9.9998903e-01, 1.1008331e-05, 9.5895747e-10],
       [1.1519188e-07, 9.9999988e-01, 2.9751215e-10],
       [9.9999988e-01, 1.1449171e-07, 7.3769697e-12],
       [3.6220763e-06, 9.9999642e-01, 1.5830848e-08],
       [1.3439801e-07, 9.9999988e-01, 3.6069387e-08],
       [8.0279879e-14, 5.7029833e-08, 1.0000000e+00],
       [2.9028129e-09, 5.2862195e-04, 9.9947137e-01],
       [5.6672822e-09, 2.8354702e-07, 9.9999976e-01],
       [1.0000000e+00, 2.8132265e-08, 1.3500150e-11],
       [9.9999988e-01, 1.0666896e-07, 3.1796215e-09],
       [1.2634167e-08, 1.0000000e+00, 3.6338663e-10]], dtype=float32)

In [54]:
import numpy as np

# 랜덤시드 고정시키기
np.random.seed(3)

from keras.preprocessing.image import ImageDataGenerator

# 데이터셋 불러오기
# train_datagen = ImageDataGenerator(rescale=1./255)

# train_generator = train_datagen.flow_from_directory(
#         'warehouse/hard_handwriting_shape/train',
#         target_size=(24, 24),
#         batch_size=3,
#         class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'C:/임시/RData/test/',
        target_size=(24, 24),    
        batch_size=3,
        class_mode='categorical')

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D



Found 15 images belonging to 3 classes.


In [55]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(24,24,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit_generator(
        train_generator,
        steps_per_epoch=15,
        epochs=200,
        validation_data=test_generator,
        validation_steps=5)


Epoch 1/200
15/15 [==============================] - 0s 19ms/step - loss: 1.0698 - accuracy: 0.5333 - val_loss: 1.2743 - val_accuracy: 0.2667
Epoch 2/200
15/15 [==============================] - 0s 12ms/step - loss: 0.2356 - accuracy: 0.9111 - val_loss: 3.1701 - val_accuracy: 0.3333
Epoch 3/200
15/15 [==============================] - 0s 15ms/step - loss: 0.0258 - accuracy: 1.0000 - val_loss: 5.1401 - val_accuracy: 0.3333
Epoch 4/200
15/15 [==============================] - 0s 13ms/step - loss: 0.0066 - accuracy: 1.0000 - val_loss: 5.1414 - val_accuracy: 0.3333
Epoch 5/200
15/15 [==============================] - 0s 12ms/step - loss: 8.9146e-04 - accuracy: 1.0000 - val_loss: 5.4967 - val_accuracy: 0.3333
Epoch 6/200
15/15 [==============================] - 0s 12ms/step - loss: 4.4974e-04 - accuracy: 1.0000 - val_loss: 5.6936 - val_accuracy: 0.3333
Epoch 7/200
15/15 [==============================] - 0s 12ms/step - loss: 3.7288e-04 - accuracy: 1.0000 - val_loss: 5.7571 - val_accuracy: 0

Epoch 57/200
15/15 [==============================] - 0s 12ms/step - loss: 4.7419e-07 - accuracy: 1.0000 - val_loss: 6.5491 - val_accuracy: 0.4000
Epoch 58/200
15/15 [==============================] - 0s 12ms/step - loss: 4.5564e-07 - accuracy: 1.0000 - val_loss: 6.5643 - val_accuracy: 0.4000
Epoch 59/200
15/15 [==============================] - 0s 12ms/step - loss: 4.2915e-07 - accuracy: 1.0000 - val_loss: 6.5746 - val_accuracy: 0.4000
Epoch 60/200
15/15 [==============================] - 0s 12ms/step - loss: 4.1326e-07 - accuracy: 1.0000 - val_loss: 6.5751 - val_accuracy: 0.4000
Epoch 61/200
15/15 [==============================] - 0s 12ms/step - loss: 4.0001e-07 - accuracy: 1.0000 - val_loss: 6.5834 - val_accuracy: 0.4000
Epoch 62/200
15/15 [==============================] - 0s 12ms/step - loss: 3.8677e-07 - accuracy: 1.0000 - val_loss: 6.6225 - val_accuracy: 0.4000
Epoch 63/200
15/15 [==============================] - 0s 12ms/step - loss: 3.7087e-07 - accuracy: 1.0000 - val_loss: 6

15/15 [==============================] - 0s 13ms/step - loss: 3.7087e-08 - accuracy: 1.0000 - val_loss: 7.2783 - val_accuracy: 0.4000
Epoch 168/200
15/15 [==============================] - 0s 12ms/step - loss: 3.7087e-08 - accuracy: 1.0000 - val_loss: 7.2800 - val_accuracy: 0.4000
Epoch 169/200
15/15 [==============================] - 0s 12ms/step - loss: 3.7087e-08 - accuracy: 1.0000 - val_loss: 7.2769 - val_accuracy: 0.4000
Epoch 170/200
15/15 [==============================] - 0s 12ms/step - loss: 3.4438e-08 - accuracy: 1.0000 - val_loss: 7.2789 - val_accuracy: 0.4000
Epoch 171/200
15/15 [==============================] - 0s 12ms/step - loss: 3.4438e-08 - accuracy: 1.0000 - val_loss: 7.2795 - val_accuracy: 0.4000
Epoch 172/200
15/15 [==============================] - 0s 12ms/step - loss: 3.4438e-08 - accuracy: 1.0000 - val_loss: 7.2787 - val_accuracy: 0.4000
Epoch 173/200
15/15 [==============================] - 0s 12ms/step - loss: 3.4438e-08 - accuracy: 1.0000 - val_loss: 7.2833 -

In [56]:
scores = model.evaluate_generator(
            test_generator, 
            steps = 5)

In [57]:
output = model.predict_generator(
            test_generator, 
            steps = 5)

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

print(output)

[[0.000 0.000 1.000]
 [0.000 0.000 1.000]
 [0.000 0.000 1.000]
 [0.000 0.002 0.998]
 [0.000 0.000 1.000]
 [0.000 0.001 0.999]
 [0.000 0.000 1.000]
 [0.000 0.000 1.000]
 [0.000 0.000 1.000]
 [0.017 0.014 0.970]
 [0.000 0.000 1.000]
 [0.767 0.000 0.233]
 [0.538 0.015 0.447]
 [0.000 0.000 1.000]
 [0.000 0.000 1.000]]


In [58]:
scores

[7.343306541442871, 0.4000000059604645]

In [59]:
newp = 'C:/임시/RData/handwriting_shape/'

In [61]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D

# 랜덤시드 고정시키기
np.random.seed(3)

# 데이터셋 불러오기
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        newp+'train',
        target_size=(24, 24),
        batch_size=3,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        newp+'test2',
        target_size=(24, 24),    
        batch_size=3,
        class_mode='categorical')

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(24,24,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit_generator(
        train_generator,
        steps_per_epoch=15,
        epochs=200,
        validation_data=test_generator,
        validation_steps=5)

scores = model.evaluate_generator(
            test_generator, 
            steps = 5)

print(scores[1])

output = model.predict_generator(
            test_generator, 
            steps = 5)

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

print(output)

Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.
Epoch 1/200
15/15 [==============================] - 0s 19ms/step - loss: 1.1050 - accuracy: 0.5778 - val_loss: 1.1678 - val_accuracy: 0.2000
Epoch 2/200
15/15 [==============================] - 0s 12ms/step - loss: 0.3798 - accuracy: 0.8222 - val_loss: 2.7675 - val_accuracy: 0.3333
Epoch 3/200
15/15 [==============================] - 0s 13ms/step - loss: 0.0668 - accuracy: 1.0000 - val_loss: 3.7908 - val_accuracy: 0.4000
Epoch 4/200
15/15 [==============================] - 0s 12ms/step - loss: 0.0084 - accuracy: 1.0000 - val_loss: 4.6830 - val_accuracy: 0.4000
Epoch 5/200
15/15 [==============================] - 0s 12ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 5.2014 - val_accuracy: 0.4000
Epoch 6/200
15/15 [==============================] - 0s 12ms/step - loss: 7.8160e-04 - accuracy: 1.0000 - val_loss: 5.4093 - val_accuracy: 0.4000
Epoch 7/200
15/15 [==============================] - 0s 12ms/ste

Epoch 112/200
15/15 [==============================] - 0s 12ms/step - loss: 3.9736e-07 - accuracy: 1.0000 - val_loss: 8.0540 - val_accuracy: 0.4000
Epoch 113/200
15/15 [==============================] - 0s 12ms/step - loss: 3.8677e-07 - accuracy: 1.0000 - val_loss: 8.0641 - val_accuracy: 0.4000
Epoch 114/200
15/15 [==============================] - 0s 11ms/step - loss: 3.7617e-07 - accuracy: 1.0000 - val_loss: 8.0705 - val_accuracy: 0.4000
Epoch 115/200
15/15 [==============================] - 0s 11ms/step - loss: 3.6822e-07 - accuracy: 1.0000 - val_loss: 8.0705 - val_accuracy: 0.4000
Epoch 116/200
15/15 [==============================] - 0s 11ms/step - loss: 3.6028e-07 - accuracy: 1.0000 - val_loss: 8.0750 - val_accuracy: 0.4000
Epoch 117/200
15/15 [==============================] - 0s 12ms/step - loss: 3.5498e-07 - accuracy: 1.0000 - val_loss: 8.0913 - val_accuracy: 0.4000
Epoch 118/200
15/15 [==============================] - 0s 12ms/step - loss: 3.5233e-07 - accuracy: 1.0000 - val_

In [62]:
from keras.preprocessing.image import array_to_img, img_to_array, load_img
np.random.seed(5)

data_aug_gen = ImageDataGenerator(rescale=1./255, 
                                  rotation_range=15,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  shear_range=0.5,
                                  zoom_range=[0.8, 2.0],
                                  horizontal_flip=True,
                                  vertical_flip=True,
                                  fill_mode='nearest')


In [ ]:
img = load_img(newp+'train/triangle/triangle001.png')
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

i = 0
for batch in train_datagen.flow(x, batch_size=1, save_to_dir='warehouse/preview', save_prefix='tri', save_format='png'):
    i += 1
    if i > 30: 
        break
